Import the necessary components. Some of them are new for this week, they will be discussed later in the notebook.

In [1]:
!pip install accelerate -U
!pip install transformers[torch]

In [2]:
pip install datasets

In [26]:
from transformers import AutoModelForSeq2SeqLM, T5Tokenizer, AutoModelForQuestionAnswering, EarlyStoppingCallback, Seq2SeqTrainingArguments,Trainer, DataCollatorForSeq2Seq, TrainingArguments
import torch
import time
import pandas as pd

import numpy as np
import re
from datasets import  load_from_disk,load_dataset


<a name='1.2'></a>
### 1.2 - Load Dataset and LLM


In [4]:
data_path = '/content/datanewWithoutNoRealplays.csv'
data = pd.read_csv(data_path)

data['Context'] = "الاعلان:\n" + data["post comment"] + "\n التعليق:" + data["Comment"]


In [5]:
data

,Comment,Reply,post comment,Author,Context
0,بالتوفيق ان شاءالله يارب,@رضا من الله يكفيني شكرا لحضرتك و احنا حقيقى س...,ادفع اقساطك من مكانك بڤودافون كاش وليك ٢٠٪ كاش...,رضا من الله يكفيني,الاعلان:\nادفع اقساطك من مكانك بڤودافون كاش ول...
1,رمضان المبارك ://./+40120 .,اهلا بيك يسعدنا رضاك عن الخدمه ونتمني دايما نك...,ادفع اقساطك من مكانك بڤودافون كاش وليك ٢٠٪ كاش...,Gideon Michael,الاعلان:\nادفع اقساطك من مكانك بڤودافون كاش ول...
2,://./+40120 .,اهلا بحضرتك من فضلك استفسار حضرتك بخصوص شحن كر...,ادفع اقساطك من مكانك بڤودافون كاش وليك ٢٠٪ كاش...,Gideon Michael,الاعلان:\nادفع اقساطك من مكانك بڤودافون كاش ول...
3,عاوز الغي خدمه الانترنت لو سمحت,احنا حقيقى فى خدمتك هل محتاج اى مساعدة تانية م...,ادفع اقساطك من مكانك بڤودافون كاش وليك ٢٠٪ كاش...,Gattuso Go,الاعلان:\nادفع اقساطك من مكانك بڤودافون كاش ول...
4,احسن شركه فودافون في مصر,فى خدمتك دايما 12,ادفع اقساطك من مكانك بڤودافون كاش وليك ٢٠٪ كاش...,احمد المصري,الاعلان:\nادفع اقساطك من مكانك بڤودافون كاش ول...
...,...,...,...,...,...
23243,لو سمحت معايا خط متسجل بإسمي عاوز أتنازل عنه ل...,اتشرفنا بيك وموجودين لمساعدتك اي وقت27,ليك ضعف الميموري زيادة بنفس السعر لما تشتريGal...,محمد محمود,الاعلان:\nليك ضعف الميموري زيادة بنفس السعر لم...
23244,لو سمحتواردوا عالخاص,اهلا وسهلا تم الرد27,ليك ضعف الميموري زيادة بنفس السعر لما تشتريGal...,ستنا زينب,الاعلان:\nليك ضعف الميموري زيادة بنفس السعر لم...
23245,لو سمحتواردوا عالخاص,اهلا وسهلا تم الرد27,ليك ضعف الميموري زيادة بنفس السعر لما تشتريGal...,ستنا زينب,الاعلان:\nليك ضعف الميموري زيادة بنفس السعر لم...
23246,لو سمحتواردوا عالخاص,اهلا وسهلا تم الرد27,ليك ضعف الميموري زيادة بنفس السعر لما تشتريGal...,ستنا زينب,الاعلان:\nليك ضعف الميموري زيادة بنفس السعر لم...


In [6]:
dataset_dict = {
    'question': data["Context"].tolist(),
    'answer': data['Reply'].tolist()
}
dataset_dict

{'question': ['الاعلان:\nادفع اقساطك من مكانك بڤودافون كاش وليك ٢٠٪ كاش باكوقتك مش ببلاش مفيش اسرع من #ڤودافون_كاش\n التعليق:بالتوفيق ان شاءالله يارب',
  'الاعلان:\nادفع اقساطك من مكانك بڤودافون كاش وليك ٢٠٪ كاش باكوقتك مش ببلاش مفيش اسرع من #ڤودافون_كاش\n التعليق:رمضان المبارك ://./+40120 . ',
  'الاعلان:\nادفع اقساطك من مكانك بڤودافون كاش وليك ٢٠٪ كاش باكوقتك مش ببلاش مفيش اسرع من #ڤودافون_كاش\n التعليق:://./+40120 . ',
  'الاعلان:\nادفع اقساطك من مكانك بڤودافون كاش وليك ٢٠٪ كاش باكوقتك مش ببلاش مفيش اسرع من #ڤودافون_كاش\n التعليق:عاوز الغي خدمه الانترنت لو سمحت',
  'الاعلان:\nادفع اقساطك من مكانك بڤودافون كاش وليك ٢٠٪ كاش باكوقتك مش ببلاش مفيش اسرع من #ڤودافون_كاش\n التعليق:احسن شركه فودافون في مصر',
  'الاعلان:\nادفع اقساطك من مكانك بڤودافون كاش وليك ٢٠٪ كاش باكوقتك مش ببلاش مفيش اسرع من #ڤودافون_كاش\n التعليق:·تمم',
  'الاعلان:\nادفع اقساطك من مكانك بڤودافون كاش وليك ٢٠٪ كاش باكوقتك مش ببلاش مفيش اسرع من #ڤودافون_كاش\n التعليق:انا مصري بجد ://./+40120 . ',
  'الاعلان:\nادفع اقساطك

In [7]:
from datasets import DatasetDict, Dataset

In [8]:

hf_dataset = Dataset.from_dict(dataset_dict)
hf_dataset


Dataset({
    features: ['question', 'answer'],
    num_rows: 23248
})

In [9]:
dataset_size = len(data)
train_size = int(0.8 * dataset_size)
val_size = int(0.1 * dataset_size)
test_size = dataset_size - train_size - val_size

In [10]:
train_dataset = hf_dataset.select(range(train_size))
validation_dataset = hf_dataset.select(range(train_size, train_size + val_size))
test_dataset = hf_dataset.select(range(train_size + val_size, len(hf_dataset)))

In [11]:
dataset = DatasetDict({
    "train": train_dataset,
    "validation": validation_dataset,
    "test": test_dataset
})

In [12]:
for split_name, split_dataset in dataset.items():
    print(f"{split_name}: {split_dataset}")

train: Dataset({
    features: ['question', 'answer'],
    num_rows: 18598
})
validation: Dataset({
    features: ['question', 'answer'],
    num_rows: 2324
})
test: Dataset({
    features: ['question', 'answer'],
    num_rows: 2326
})


In [13]:

features = dataset["train"].features
dataset = dataset.map(lambda example: {k: v for k, v in example.items() if k in features})

Map:   0%|          | 0/18598 [00:00<?, ? examples/s]

Map:   0%|          | 0/2324 [00:00<?, ? examples/s]

Map:   0%|          | 0/2326 [00:00<?, ? examples/s]

In [14]:
import gc
gc.collect()

96

In [15]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.cuda.set_device(device)

In [27]:
model_name = "UBC-NLP/AraT5v2-base-1024"
original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer)

tokenizer_config.json:   0%|          | 0.00/2.37k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/2.35M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.40M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [29]:
def preprocess_function(example,padding="max_length"):
    # add prefix to the input for t5
    inputs = example["question"]

    # tokenize inputs
    model_inputs = tokenizer(inputs, max_length=512, padding=padding, truncation=True)

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(text_target=example["answer"], max_length=512, padding=padding, truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['question', 'answer'])
print(f"Keys of tokenized dataset: {list(tokenized_datasets['train'].features)}")


Map:   0%|          | 0/18598 [00:00<?, ? examples/s]

Map:   0%|          | 0/2324 [00:00<?, ? examples/s]

Map:   0%|          | 0/2326 [00:00<?, ? examples/s]

Keys of tokenized dataset: ['input_ids', 'attention_mask', 'labels']


Check the shapes of all three parts of the dataset:

In [30]:
print(f"Shapes of the datasets:")
print(f"Training: {tokenized_datasets['train'].shape}")
print(f"Validation: {tokenized_datasets['validation'].shape}")
print(f"Test: {tokenized_datasets['test'].shape}")


print(tokenized_datasets)

Shapes of the datasets:
Training: (18598, 3)
Validation: (2324, 3)
Test: (2326, 3)
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 18598
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 2324
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 2326
    })
})


<a name='2.2'></a>
### 2.2 - Fine-Tune the Model with the Preprocessed Dataset



In [41]:
output_dir = "./result"

training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=5e-5, # Higher learning rate than full fine-tuning.
    num_train_epochs=2,
    # max_grad_norm=0.1,
    logging_steps=500,
    # eval_steps=500,
    # save_strategy= "steps",
    evaluation_strategy="steps",
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    # save_steps=1024,
    # fp16=True,
    save_total_limit=2,

)

trainer = Trainer(
    model=original_model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [42]:
import gc
gc.collect()

170

In [43]:
trainer.train()

Step,Training Loss,Validation Loss
500,3.600100,2.258951
1000,2.538600,1.697983
1500,2.343200,1.372228
2000,1.961000,1.152951
2500,1.695500,0.943562
3000,1.622800,0.815814
3500,1.503800,0.714572
4000,1.444100,0.650358
4500,1.230500,0.603480
5000,1.199300,0.575382


Step,Training Loss,Validation Loss
500,3.600100,2.258951
1000,2.538600,1.697983
1500,2.343200,1.372228
2000,1.961000,1.152951
2500,1.695500,0.943562
3000,1.622800,0.815814
3500,1.503800,0.714572
4000,1.444100,0.650358
4500,1.230500,0.603480
5000,1.199300,0.575382


TrainOutput(global_step=9300, training_loss=1.527244380827873, metrics={'train_runtime': 5089.5278, 'train_samples_per_second': 7.308, 'train_steps_per_second': 1.827, 'total_flos': 3.232225216482509e+16, 'train_loss': 1.527244380827873, 'epoch': 2.0})

In [35]:

model_path= "medhat6969"
trainer.model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

('medhat6969/tokenizer_config.json',
 'medhat6969/special_tokens_map.json',
 'medhat6969/spiece.model',
 'medhat6969/added_tokens.json')